<a href="https://colab.research.google.com/github/freentour/AIFFEL_quest/blob/main/Python_quest/Advanced/RPG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
# Character 클레스 : 김원영님 작업
class Character:
    def __init__(self, name, level=0, hp=0, attack=0, defense=0):
        self.name = name
        self.level = level
        self.hp = hp
        self.attack = attack
        self.defense = defense

    def __str__(self):
        return f"이름: {self.name}, 레벨: {self.level}, 체력: {self.hp}, 공격력: {self.attack}, 방어력: {self.defense}"

    def is_alive(self):
        return self.hp > 0

    # def attack_target(self, target):
    #     damage = max(self.attack - target.defense, 0)
    #     target.take_damage(damage)
    #     print(f'{self.name} 이(가) {target.name} 에게 {damage} 데미지의 공격')
    #     print(f'{target.name} 은(는) {damage}만큼의 타격을 입었다. {target.name}의 남은 체력: {target.hp}')

    # def take_damage(self, damage):
    #     self.hp -= damage

    def attack_target(self, target):
        damage = random.randint(1, self.attack)
        target.take_damage(damage)
        print(f'{self.name} 이(가) {target.name} 에게 {damage} 데미지의 공격')
        print(f'{target.name} 은(는) {damage}만큼의 타격을 입었다. {target.name}의 남은 체력: {target.hp}')

    def take_damage(self, damage):
        if self.defense >= damage:
            pass    # 체력 감소 없음
        else:
            down = damage - self.defense
            if down >= self.hp:
                self.hp = 0
            else:
                self.hp -= down


In [56]:
# Player 클래스 : 이유진님 작업
class Player(Character):
    def __init__(self, name):  # name을 입력받고
        super().__init__(name, level=1, hp=100, attack=25, defense=5)  # 부모 클래스의 __init__ 호출
        self.exp = 0    # 경험치 초기화

    def __str__(self):
        base_str = super().__str__()  # 부모 클래스의 __str__ 메서드 호출
        return f"{base_str}, 경험치: {self.exp}"  # 추가적인 정보 출력

    def gain_experience(self, exp):  # 경험치를 받아서 더해주기
        print(f"{self.name}은 {exp}만큼의 경험치를 얻었다.")
        self.exp += exp
        self.level_up()             # 자동으로 레벨업 여부 결정하도록 메소드 실행

    def level_up(self):
        while True :                 # 무한루프를 가동
            if self.exp >= 50:       # 경험치가 50이 넘으면 레벨이 올라감
                self.level += 1
                self.attack += 10
                self.defense += 5
                self.exp -= 50
                print(f"{self.name}이 레벨 1 올라서 레벨 {self.level}가 되었습니다!")
                # print(self.__str__())
            else:                    # 겸험치가 50 미만일 경우 무한루프 종료
                print(self.__str__())
                break

    def __enter__(self):
        print(f"환영합니다. {self.name}!\n")
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        print(f"{self.name}, 플레이 해주셔서 감사합니다!")

In [52]:
# Monster 클래스 : 김소영님 작업
import random

class Monster(Character):
    def __init__(self, name, level):                          # name과 level 입력받기
        super().__init__(name, level)
        self.hp = self.stat(0, 25)                          # 체력 : 0-25로 설정
        self.attack = self.stat(5, 10)                      # 공격력 : 5-10로 설정
        self.defense = self.stat(1, 5)                      # 방어력 : 1-5로 설정
        # super().__init__(name, level)

    def __str__(self):
        return f"{super().__str__()}"                         # 부모 클래스의 __str__ 호출

    def stat(self, min_value, max_value):
        return random.randint(min_value, max_value) * self.level   # (hp, attack, defense) 범위내의 랜덤값 * level

m = Monster('고블린', 1)
print(m)

이름: 고블린, 레벨: 1, 체력: 13, 공격력: 5, 방어력: 3


In [53]:
# 전투 함수 : 김원영님 작업
def battle(player, monster):
    print("전투에 돌입합니다. 현재 상태")
    print(f'{player.name} (현재 체력: {player.hp}, 공격력: {player.attack}, 방어력: {player.defense})')
    print(f'{monster.name} (현재 체력: {monster.hp}, 공격력: {monster.attack}, 방어력: {monster.defense})')
    print("--------------------------------------------------")

    while player.is_alive() and monster.is_alive():
        player.attack_target(monster)
        if monster.is_alive():
            monster.attack_target(player)
        print("--------------------------------------------------")

    if player.is_alive():
        player.gain_experience(30)
        print(f"전투 승리!")
        print("--------------------------------------------------")
    else:
        print("전투 패배..")
        print("--------------------------------------------------")


In [54]:
# 몬스터 생성을 위한 제너레이터 함수 : 김소영님 작업
"""
몬스터 이름과 레벨이 주어지면 1~3개의 몬스터 객체가 랜덤하게 생성되는 제너레이터 함수

사용법 : gen = generate_monster(name, level)
- name : 고블린/트롤/늑대인간/뱀파이어/피카츄 중 하나.
- level : 1~5 (스테이지 갯수와 동일하게 맞춤)

참고사항
- 제너레이터이므로 next(gen) 호출했을 때 순서대로 다음 몬스터 객체가 리턴됨.
- 제너레이터의 마지막 요소까지 호출한 이후에 다시 next(gen)이 호출되면 StopIteration 예외 발생!
- 제너레이터이므로 next() 함수 대신 for 반복문을 사용해 각 객체를 순회하면서 처리해도 됨. (이 경우에는 StopIteration 발생하지 않음!)

수정 내역
- 2024.05.29(수) 최초 생성. (김소영)
"""
def generate_monster(name, level): # 몬스터 이름과 레벨 인수로 받음
    num_monsters = random.randint(1, 3) # 1-3사이의 랜덤한 monster 생성
    print(f"{num_monsters}마리의 {name}을 마주쳤습니다!")

    # monsters = [] #생성한 monster 저장할 곳
    for i in range(num_monsters): # num_mosters만큼 반복해서 monster 생성
       monster =  Monster(name, level)
       yield monster


In [58]:
# from character import *
# from battle import *

# main 함수 : 이유진님, 김주현님 공동 작업
def main():
  stage_list = [1, 2, 3, 4, 5]
  character_list = ['용사','마법사','워리어','신부','테이머']
	# monster_dict : key-스테이지 번호, value-몬스터 이름, 몬스터의 레벨은 key인 스테이지 번호와 동일하게 함.
  monster_dict = {1:'고블린', 2:'트롤', 3:'늑대인간', 4:'뱀파이어', 5:'피카츄'}

  print("게임을 시작합니다.")
  print("\n")

  while True:
    character_name = input("캐릭터 이름을 입력하세요. (용사/마법사/워리어/신부/테이머) : ")
    if character_name not in character_list:
      print("선택할 수 없는 캐릭터입니다. 다시 입력해주세요.")
    else:
      break

  # 게임 중 플레이어 생존 여부를 확인하기 위한 플래그 변수
  is_alive_player = 'y'

  with Player(character_name) as player:
    # print(player)
    print("="*50)
    print("\n")

    for stage_no in stage_list:
      print("-"*50)
      print(f"{stage_no}번째 스테이지 시작")
      print("-"*50)

      mon_name = monster_dict[stage_no]
      mon_level = stage_no

      gen = generate_monster(mon_name, mon_level)
      # 허걱!! list 함수를 돌고 나니 제너레이터는 이미 순회를 끝!
      # 따라서, 이 상태에서 nex(gen) 호출하면 곧바로 StopIteration 예외가 발생함!
      # monsters = list(gen)
      # print(f"{len(monsters)}마리의 {mon_name}을 마주쳤습니다!")


      while is_alive_player == 'y':
        try:
          monster = next(gen)
        except StopIteration as e:
          print(f"{stage_no}번째 스테이지 종료")
          print("-"*50)
          print("\n")
          break
        else:
          # 전투 실행
          battle(player, monster)

          if not player.is_alive():
            print(f"{player.name}이 사망했습니다.")
            is_alive_player = 'n'

      if not player.is_alive():
        break

    print("\n")

    if player.is_alive():
      print("축하합니다! 모든 스테이지를 완료하였습니다.")

    print("게임을 종료합니다.")
    print("\n")

if __name__ == '__main__':
  main()


게임을 시작합니다.


캐릭터 이름을 입력하세요. (용사/마법사/워리어/신부/테이머) : 마법사
환영합니다. 마법사!



--------------------------------------------------
1번째 스테이지 시작
--------------------------------------------------
2마리의 고블린을 마주쳤습니다!
전투에 돌입합니다. 현재 상태
마법사 (현재 체력: 100, 공격력: 25, 방어력: 5)
고블린 (현재 체력: 17, 공격력: 5, 방어력: 3)
--------------------------------------------------
마법사 이(가) 고블린 에게 12 데미지의 공격
고블린 은(는) 12만큼의 타격을 입었다. 고블린의 남은 체력: 8
고블린 이(가) 마법사 에게 1 데미지의 공격
마법사 은(는) 1만큼의 타격을 입었다. 마법사의 남은 체력: 100
--------------------------------------------------
마법사 이(가) 고블린 에게 3 데미지의 공격
고블린 은(는) 3만큼의 타격을 입었다. 고블린의 남은 체력: 8
고블린 이(가) 마법사 에게 5 데미지의 공격
마법사 은(는) 5만큼의 타격을 입었다. 마법사의 남은 체력: 100
--------------------------------------------------
마법사 이(가) 고블린 에게 16 데미지의 공격
고블린 은(는) 16만큼의 타격을 입었다. 고블린의 남은 체력: 0
--------------------------------------------------
마법사은 30만큼의 경험치를 얻었다.
이름: 마법사, 레벨: 1, 체력: 100, 공격력: 25, 방어력: 5, 경험치: 30
전투 승리!
--------------------------------------------------
전투에 돌입합니다. 현재 상태
마법사 (현재 체력: 100, 공격력: 25, 방어력: 5)
고블린 (현재 체